In [ ]:
with open("16/input.txt") as f:
    data = f.read()
print(data)

In [ ]:
rows = data.splitlines()
rows

In [ ]:
def print_map(map: list[str]):
    for row in map:
        print(row)

In [ ]:
def mark(visited_map: list[str], position: tuple[int, int]):
    y, x = position
    row = visited_map[y]
    visited_map[y] = row[:x] + "x" + row[x+1:]
    return visited_map

In [ ]:
from enum import Enum

class Direction(tuple[int, int], Enum):
    UP = (-1, 0)
    RIGHT = (0, 1)
    DOWN = (1, 0)
    LEFT = (0, -1)

In [ ]:
def transition(direction: Direction, char: str) -> tuple[Direction, Direction | None]:
    match (direction, char):
        case (_, ".") | (Direction.LEFT | Direction.RIGHT, "-") | (Direction.UP | Direction.DOWN, "|"):
            return direction, None

        case Direction.LEFT | Direction.RIGHT, "|":
            return Direction.UP, Direction.DOWN
        

        case Direction.UP | Direction.DOWN, "-":
            return Direction.LEFT, Direction.RIGHT
        
        case _, "\\":
            match direction:
                case Direction.UP:
                    return Direction.LEFT, None
                
                case Direction.RIGHT:
                    return Direction.DOWN, None
                
                case Direction.DOWN:
                    return Direction.RIGHT, None
                
                case Direction.LEFT:
                    return Direction.UP, None
        
        case _, "/":
            match direction:
                case Direction.UP:
                    return Direction.RIGHT, None
                
                case Direction.RIGHT:
                    return Direction.UP, None
                
                case Direction.DOWN:
                    return Direction.LEFT, None
                    
                case Direction.LEFT:
                    return Direction.DOWN, None

        case _:
            raise ValueError(f"The combination of direction: {direction} and char: {char} is not handled.")


In [ ]:

def move(map: list[str], position: tuple[int, int], direction: Direction) -> list[tuple[tuple[int, int], Direction]]:
    yd, xd = direction
    yc, xc = position

    current_position = (yd + yc, xd + xc)
    yc, xc = current_position

    if yc < 0 or yc >= len(map) or xc < 0 or xc >= len(map[0]):
        return []

    symbol = map[current_position[0]][current_position[1]]

    new_directions = transition(direction, symbol)

    return [(current_position, d) for d in new_directions if d is not None]

def count_energized(map: list[str], initial_position: tuple[int, int], initial_direction: Direction):
    positions = [
        (initial_position, initial_direction)
    ]

    visited_with_dirs = set()

    while(len(positions) > 0):
        (position, direction), *positions = positions
        if (position, direction) not in visited_with_dirs:
            new_positions = move(map, position, direction)
            positions.extend(new_positions)
            visited_with_dirs.add((position, direction))

    return len(set([
        (y, x) for ((y,x), _) in visited_with_dirs
        if y >= 0 and y < len(map) and x >= 0 and x < len(map[0])
    ]))

In [ ]:
count_energized(rows, (0, -1), Direction.RIGHT)

## Part 2

In [ ]:
initial_starting_positions = [
    ((y, -1), Direction.RIGHT)
    for y in range(len(rows))
] + [
    ((y, len(rows[0])), Direction.LEFT)
    for y in range(len(rows))
] + [
    ((-1, x), Direction.DOWN)
    for x in range(len(rows[0]))
] + [
    ((len(rows), x), Direction.UP)
    for x in range(len(rows[0]))
]
initial_starting_positions

In [ ]:
from functools import reduce, partial
from itertools import starmap

reduce(max, starmap(partial(count_energized, rows), initial_starting_positions))